In [ ]:
!pip install -q transformers==4.44.0 albumentations==1.4.8 timm==0.9.2
!pip uninstall -y albumentations albucore
!pip install albumentations==1.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.4 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from transformers import SegformerForSemanticSegmentation
import matplotlib.pyplot as plt

In [1]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

ROOT = "/content/drive/MyDrive/Kuliah/Skripsi S1"

TRAIN_IMAGES_DIR = os.path.join(ROOT, "train/images")
TRAIN_MASKS_DIR  = os.path.join(ROOT, "train/targets")
TEST_IMAGES_DIR  = os.path.join(ROOT, "test/images")
TEST_MASKS_DIR   = os.path.join(ROOT, "test/targets")

IMG_SIZE    = 512
BATCH_SIZE  = 8
NUM_EPOCHS  = 20
LEARNING_RATE = 1e-4

NameError: name 'torch' is not defined

In [ ]:
class BuildingDataset(Dataset):
    def __init__(self, image_ids, images_dir, masks_dir, transform=None, only_pre=True):
        self.image_ids = image_ids
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_path = os.path.join(self.images_dir, image_id)

        image_bgr = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if image_bgr is None:
            raise RuntimeError(f"Cannot read image: {img_path}")
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

        base = image_id
        if base.endswith(".png"):
            base = base[:-4]
        base_clean = base.replace(" (1)", "")
        mask_name = base_clean + "_target.png"
        mask_path = os.path.join(self.masks_dir, mask_name)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise RuntimeError(f"Cannot read mask: {mask_path}")

        mask_bin = (mask > 0).astype("float32")

        if self.transform is not None:
            augmented = self.transform(image=image_rgb, mask=mask_bin)
            image = augmented["image"]
            mask  = augmented["mask"].unsqueeze(0)
        else:
            image = ToTensorV2()(image=image_rgb)["image"]
            mask  = torch.from_numpy(mask_bin).unsqueeze(0)

        return image, mask

In [ ]:
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

def get_image_ids(images_dir):
    return sorted([
        f for f in os.listdir(images_dir)
        if os.path.isfile(os.path.join(images_dir, f))
        and f.endswith(".png")
        and "_pre_disaster" in f
    ])

train_ids = get_image_ids(TRAIN_IMAGES_DIR)
train_ids, val_ids = train_test_split(
    train_ids,
    test_size=0.10,
    random_state=42,
    shuffle=True
)
test_ids  = get_image_ids(TEST_IMAGES_DIR)

print("Train images:", len(train_ids))
print("Val images:", len(val_ids))
print("Test images:", len(test_ids))

train_dataset = BuildingDataset(train_ids, TRAIN_IMAGES_DIR, TRAIN_MASKS_DIR, transform=train_transform)
val_dataset   = BuildingDataset(val_ids,   TRAIN_IMAGES_DIR,   TRAIN_MASKS_DIR,   transform=val_transform)
test_dataset  = BuildingDataset(test_ids,  TEST_IMAGES_DIR,  TEST_MASKS_DIR,  transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
def iou_score_from_logits(logits, targets, eps=1e-7):
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).float()
    targets = targets.float()

    inter = (preds * targets).sum(dim=(2,3))
    union = preds.sum(dim=(2,3)) + targets.sum(dim=(2,3)) - inter

    iou = (inter + eps) / (union + eps)
    return iou.mean().item()


def dice_score_from_logits(logits, targets, eps=1e-7):
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).float()
    targets = targets.float()

    inter = (preds * targets).sum(dim=(2,3))
    union = preds.sum(dim=(2,3)) + targets.sum(dim=(2,3))

    dice = (2*inter + eps) / (union + eps)
    return dice.mean().item()

In [ ]:
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/segformer-b0-finetuned-ade-512-512",
    num_labels=1,
    ignore_mismatched_sizes=True
).to(DEVICE)

# just to see size
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"SegFormer-B0 params: {total_params/1e6:.2f}M (trainable: {trainable_params/1e6:.2f}M)")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/15.0M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SegFormer-B0 params: 3.71M (trainable: 3.71M)


In [ ]:
bce_loss = nn.BCEWithLogitsLoss()

def dice_loss_from_logits(logits, targets, eps=1e-7):
    probs = torch.sigmoid(logits)
    targets = targets.float()
    inter = (probs * targets).sum(dim=(2,3))
    union = probs.sum(dim=(2,3)) + targets.sum(dim=(2,3))
    dice = (2*inter + eps) / (union + eps)
    return 1 - dice.mean()

def seg_loss(logits, targets):
    return bce_loss(logits, targets) + dice_loss_from_logits(logits, targets)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)


In [12]:
best_iou = 0.0
CHECKPOINT_DIR = os.path.join(ROOT, "checkpoints")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


In [ ]:

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss = 0.0
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch} - Training", leave=False)

    for step, (images, masks) in enumerate(train_loader_tqdm, start=1):
        images = images.to(DEVICE)
        masks  = masks.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(pixel_values=images)
        logits  = outputs.logits

        logits_upsampled = F.interpolate(
            logits,
            size=masks.shape[-2:],
            mode="bilinear",
            align_corners=False
        )

        loss = seg_loss(logits_upsampled, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

        if step % 10 == 0:
            train_loader_tqdm.set_postfix({"loss": f"{loss.item():.4f}"})

    train_loss /= len(train_loader.dataset)

    # val
    model.eval()
    val_loss = 0.0
    val_iou  = 0.0
    val_dice = 0.0
    val_bf   = 0.0
    n_batches = 0

    val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch} - Validation", leave=False)

    with torch.no_grad():
        for images, masks in val_loader_tqdm:
            images = images.to(DEVICE)
            masks  = masks.to(DEVICE)

            outputs = model(pixel_values=images)
            logits  = outputs.logits

            logits_upsampled = F.interpolate(
                logits,
                size=masks.shape[-2:],
                mode="bilinear",
                align_corners=False
            )

            loss = seg_loss(logits_upsampled, masks)
            val_loss += loss.item() * images.size(0)

            batch_iou  = iou_score_from_logits(logits_upsampled, masks)
            batch_dice = dice_score_from_logits(logits_upsampled, masks)

            val_iou  += batch_iou
            val_dice += batch_dice
            n_batches += 1

            val_loader_tqdm.set_postfix(
                iou=f"{batch_iou:.4f}",
                dice=f"{batch_dice:.4f}"
            )

    val_loss /= len(val_loader.dataset)
    val_iou  /= max(n_batches, 1)
    val_dice /= max(n_batches, 1)

    print(
        f"Epoch [{epoch}/{NUM_EPOCHS}] "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val IoU: {val_iou:.4f} | "
        f"Val Dice: {val_dice:.4f} | "
    )

    # save best
    if val_iou > best_iou:
        best_iou = val_iou
        best_path = os.path.join(CHECKPOINT_DIR, "SegFormerB0_best.pth")
        torch.save(model.state_dict(), best_path)
        print(f"🔥 New best SegFormer-B0 saved at epoch {epoch}! IoU = {best_iou:.4f}")

    # also save every 5 epochs
    if epoch % 5 == 0:
        save_path = os.path.join(CHECKPOINT_DIR, f"SegFormerB0_epoch{epoch}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"Model saved: {save_path}")


Epoch [1/20] Train Loss: 0.9962 | Val Loss: 0.7330 | Val IoU: 0.5135 | Val Dice: 0.6361 | 
🔥 New best SegFormer-B0 saved at epoch 1! IoU = 0.5135


Epoch [2/20] Train Loss: 0.7081 | Val Loss: 0.5687 | Val IoU: 0.5997 | Val Dice: 0.7191 | 
🔥 New best SegFormer-B0 saved at epoch 2! IoU = 0.5997


Epoch [3/20] Train Loss: 0.5869 | Val Loss: 0.5131 | Val IoU: 0.5544 | Val Dice: 0.6722 | 


Epoch [4/20] Train Loss: 0.5347 | Val Loss: 0.4974 | Val IoU: 0.5910 | Val Dice: 0.7073 | 


Epoch [5/20] Train Loss: 0.5069 | Val Loss: 0.4756 | Val IoU: 0.5895 | Val Dice: 0.7043 | 
Model saved: /content/drive/MyDrive/Kuliah/Skripsi S1/checkpoints/SegFormerB0_epoch5.pth


...

Epoch [16/20] Train Loss: 0.4285 | Val Loss: 0.4494 | Val IoU: 0.5976 | Val Dice: 0.7075 | 


Epoch [17/20] Train Loss: 0.4243 | Val Loss: 0.4417 | Val IoU: 0.6616 | Val Dice: 0.7709 | 


Epoch [18/20] Train Loss: 0.4198 | Val Loss: 0.4529 | Val IoU: 0.5862 | Val Dice: 0.6969 | 


Epoch [19/20] Train Loss: 0.4167 | Val Loss: 0.4377 | Val IoU: 0.6473 | Val Dice: 0.7556 | 


Epoch [20/20] Train Loss: 0.4126 | Val Loss: 0.4413 | Val IoU: 0.6554 | Val Dice: 0.7629 | 
Model saved: /content/drive/MyDrive/Kuliah/Skripsi S1/checkpoints/SegFormerB0_epoch20.pth


# TESTING

In [ ]:
# load best checkpoint
best_path = os.path.join(CHECKPOINT_DIR, "SegFormerB0_best.pth")
print("Loading best SegFormer-B0 from:", best_path)

model.load_state_dict(torch.load(best_path, map_location=DEVICE))
model.to(DEVICE)
model.eval()

test_iou  = 0.0
test_dice = 0.0
test_bf   = 0.0
n_batches = 0

with torch.no_grad():
    for images, masks in tqdm(test_loader, desc="Testing SegFormer-B0"):
        images = images.to(DEVICE)
        masks  = masks.to(DEVICE)

        outputs = model(pixel_values=images)
        logits  = outputs.logits

        logits_upsampled = F.interpolate(
            logits,
            size=masks.shape[-2:],
            mode="bilinear",
            align_corners=False
        )

        batch_iou  = iou_score_from_logits(logits_upsampled, masks)
        batch_dice = dice_score_from_logits(logits_upsampled, masks)

        test_iou  += batch_iou
        test_dice += batch_dice
        n_batches += 1

test_iou  /= max(n_batches, 1)
test_dice /= max(n_batches, 1)
test_bf   /= max(n_batches, 1)

print("=== TEST RESULTS (SegFormer-B0) ===")
print(f"Test IoU:  {test_iou:.4f}")
print(f"Test Dice: {test_dice:.4f}")
print(f"Test BF:   {test_bf:.4f}")


Loading best SegFormer-B0 from: /content/drive/MyDrive/Kuliah/Skripsi S1/checkpoints/SegFormerB0_best.pth


NameError: name 'test_loader' is not defined

In [ ]:
img_path = "/content/drive/MyDrive/Kuliah/Skripsi S1/test/images/hurricane-florence_00000013_pre_disaster.png"
mask_path = "/content/drive/MyDrive/Kuliah/Skripsi S1/test/targets/hurricane-florence_00000013_pre_disaster_target.png"

image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
gt_mask = (gt_mask > 0).astype("float32")


In [18]:
val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [ ]:
aug = val_transform(image=image, mask=gt_mask)

image_tensor = aug["image"].unsqueeze(0).to(DEVICE)
mask_tensor  = aug["mask"].unsqueeze(0).unsqueeze(0).to(DEVICE)


In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(pixel_values=image_tensor)
    logits  = outputs.logits

    logits_upsampled = F.interpolate(
        logits,
        size=mask_tensor.shape[-2:],
        mode="bilinear",
        align_corners=False
    )

    probs = torch.sigmoid(logits_upsampled)
    pred  = (probs > 0.5).float()


In [ ]:
iou  = iou_score_from_logits(logits_upsampled, mask_tensor)
dice = dice_score_from_logits(logits_upsampled, mask_tensor)

print("=== SINGLE IMAGE RESULTS (SegFormer-B0) ===")
print(f"IoU:  {iou:.4f}")
print(f"Dice: {dice:.4f}")


=== SINGLE IMAGE RESULTS (SegFormer-B0) ===
IoU:  0.8105
Dice: 0.8953
BF:   0.9953
